**Agents for your agents!**

There are two main ways to layer agents together into a multi-agent system:

1. Agents as Tools
2. Agent Routing

We'll discuss each in its own section below.

In [1]:
# This page will use the following imports:

from lasagna import Model, EventCallback, AgentRun
from lasagna import (
    recursive_extract_messages,
    override_system_prompt,
    flat_messages,
)
from lasagna import known_models
from lasagna.tui import tui_input_loop

import os

from dotenv import load_dotenv

We need to set up our "binder" (see the [quickstart guide](../quickstart.ipynb) for what this is).

In [2]:
load_dotenv()

if os.environ.get('OPENAI_API_KEY'):
    print('Using OpenAI')
    binder = known_models.openai_gpt_4o_binder

elif os.environ.get('ANTHROPIC_API_KEY'):
    print('Using Anthropic')
    binder = known_models.anthropic_claude_sonnet_4_binder

else:
    assert False, "Neither OPENAI_API_KEY nor ANTHROPIC_API_KEY is set! We need at least one to do this demo."

Using OpenAI


## Agents as Tools

The simplest way to combine agents is to pass one (or more) agents as "tools" to another agent.

Here is a simple example:

In [3]:
async def joke_specialist(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    """
    Use this agent when the user seems discouraged and needs to feel better.
    This tool will return the perfect joke to cheer the user up.
    """
    messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)
    messages = override_system_prompt(messages, "You are a joke-telling specialist. You always tell a joke related to the user's most recent message. Your response must contain **only** the joke.")
    new_messages = await model.run(event_callback, messages, tools=[])
    return flat_messages('joke_specialist', new_messages)


async def root_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)
    messages = override_system_prompt(messages, 'You are a generic assistant. Use your tools when necessary.')

    new_messages = await model.run(
        event_callback,
        messages,
        tools=[
            joke_specialist,   # <-- 🔨 downstream agent as a tool
        ],
    )

    return flat_messages('root_agent', new_messages)

In [4]:
await tui_input_loop(binder(root_agent))   # type: ignore[top-level-await]

>  Hi!


Hi there! How can I assist you today? 😊


>  Oh, I'm sick. :(


I'm sorry to hear that you're not feeling well. Being sick is never fun. Take it easy, stay hydrated, and rest as much as you can. Would you like me to find a joke to cheer you up a bit? Laughter can be great medicine! 😊


>  yes


joke_specialist({})
Why did the doctor carry a red pen?  
In case they needed to draw blood! -> Why did the doctor carry a red pen?  
In case they needed to draw blood!
Here's a little joke to cheer you up:  

Why did the doctor carry a red pen?  
In case they needed to draw blood!

Hope that brought a smile to your face! Feel better soon! 😊


>  exit


See the [Agents as Tools](../recipes/agents_as_tools.ipynb) recipe for another example.

## Agent Routing

TODO

See the [Agent Routing](../recipes/routing_agent.ipynb) recipe for a working example.